<a href="https://colab.research.google.com/github/yelim421/SolarPower/blob/main/G_boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Solar_Power

/content/drive/MyDrive/Solar_Power


# Library

In [ ]:
#!pip install lightgbm optuna

In [5]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 모델들, 성능 평가
# (저는 일반적으로 정형데이터로 머신러닝 분석할 때는 이 2개 모델은 그냥 돌려봅니다. 특히 RF가 테스트하기 좋습니다.)
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from lightgbm.sklearn import LGBMClassifier, LGBMRegressor

# 상관관계 분석, VIF : 다중공선성 제거
from statsmodels.stats.outliers_influence import variance_inflation_factor

# KFold(CV), partial : optuna를 사용하기 위함
from sklearn.model_selection import KFold
from functools import partial

# hyper-parameter tuning을 위한 라이브러리, optuna
import optuna

In [6]:
# flag setting
feature_reducing = False

In [7]:
# 데이터를 불러옵니다.
train = pd.read_csv('train_data_datetime.csv')
test = pd.read_csv('test_data_datetime.csv')

In [11]:
# 'DateTime' 열 및 종속 변수 'power' 제거
X = train.drop(columns=['DateTime', 'power', 'd2m', 'Temperature(C)'])

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# VIF 값이 높은 순서대로 정렬
vif_data_sorted = vif_data.sort_values(by="VIF", ascending=False)

vif_data_sorted

,Variable,VIF
12,t2m,148.244862
5,VaporPressure(hPa),68.340996
4,Humidity(%),53.076565
6,DewPointTemperature(C),41.040569
2,Hour,15.338579
11,Visibility(10m),13.239854
7,Sunshine(hr),9.240475
8,SolarRadiation(MJ/m2),7.750923
10,TotalCloudCoverage(1/10),6.584246
0,Month,6.105183
